In [1]:
import urllib.request
from bs4 import BeautifulSoup

soup = BeautifulSoup(urllib.request.urlopen("https://www.goszakup.gov.kz/ru/registry/rqc").read(),'lxml')

tbody = soup('table', {"class": "table table-bordered table-hover"})[0].find_all('tr')

for row in tbody:
    cols = row.findChildren(recursive=False)
    cols = [ele.text.strip() for ele in cols]
    print(cols)

['№', 'Наименование потенциального поставщика', 'БИН/ИИН', 'Наименование, номер и дата выдачи документа, на основании которого потенциальный поставщик включен в Перечень']
['467', 'Товарищество с ограниченной ответственностью "Эмир-Трэйд"', '160540016411', '115000002, 2099-12-31 00:00:00']
['502', 'Товарищество с ограниченной ответственностью"GLASMAN"', '070540008075', '102000012, 2099-12-31 00:00:00']
['517', 'ТОО "Гражданин"', '090540008881', '101 0 00010,']
['526', 'Товарищество с ограниченной ответственностью "ЭлтексАлатау"', '110740014027', '102900003, 2019-08-08 10:57:18']
['528', 'ТОО "KazBioServis-Astana"', '130740024484', '101000008, 2099-12-31 00:00:00']
['551', 'ТОО "ПРАГМА"', '000440001170', '107000007, 2099-12-31 00:00:00']
['553', 'ТОО "ЕСКО" по производству чулочно-носочных изделий', '060540009190', '110900004, 2019-07-04 10:57:10']
['562', 'ТОО "Азия - Софт Сервис ЛТД"', '031040000512', '102000037, 2099-12-31 00:00:00']
['574', 'Товарищество с ограниченной ответственнос

In [2]:
pip install selenium 

     |████████████████████████████████| 10.5 MB 9.5 MB/s eta 0:00:01
     |████████████████████████████████| 467 kB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 8.0 MB/s  eta 0:00:01
     |████████████████████████████████| 58 kB 3.6 MB/s eta 0:00:011
  Attempting uninstall: attrs
    Found existing installation: attrs 21.4.0
    Uninstalling attrs-21.4.0:
      Successfully uninstalled attrs-21.4.0
  Attempting uninstall: sniffio
    Found existing installation: sniffio 1.2.0
    Uninstalling sniffio-1.2.0:
      Successfully uninstalled sniffio-1.2.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [2]:
url = 'https://www.goszakup.gov.kz/ru/registry/rqc'
driver = webdriver.Chrome()  # Укажите путь к драйверу вашего браузера
driver.get(url)

[<selenium.webdriver.remote.webelement.WebElement (session="489617439f9b2b3d94e01348607f379e", element="f.86B585B87781698C1422B7CBEEBAA07C.d.5A05F80934441DABFBAD7503CDA5A29C.e.213")>]
None


In [5]:
import urllib.request
from bs4 import BeautifulSoup

soup = BeautifulSoup(driver.page_source, 'html.parser')
tbody = soup('table', {"class": "table table-bordered table-hover"})[0].find_all('tr')

for row in tbody:
    cols = row.findChildren(recursive=False)
    
    cols = [ele.text.strip() for ele in cols]
    print(cols)

['№', 'Наименование потенциального поставщика', 'БИН/ИИН', 'Наименование, номер и дата выдачи документа, на основании которого потенциальный поставщик включен в Перечень']
['467', 'Товарищество с ограниченной ответственностью "Эмир-Трэйд"', '160540016411', '115000002, 2099-12-31 00:00:00']
['502', 'Товарищество с ограниченной ответственностью"GLASMAN"', '070540008075', '102000012, 2099-12-31 00:00:00']
['517', 'ТОО "Гражданин"', '090540008881', '101 0 00010,']
['526', 'Товарищество с ограниченной ответственностью "ЭлтексАлатау"', '110740014027', '102900003, 2019-08-08 10:57:18']
['528', 'ТОО "KazBioServis-Astana"', '130740024484', '101000008, 2099-12-31 00:00:00']
['551', 'ТОО "ПРАГМА"', '000440001170', '107000007, 2099-12-31 00:00:00']
['553', 'ТОО "ЕСКО" по производству чулочно-носочных изделий', '060540009190', '110900004, 2019-07-04 10:57:10']
['562', 'ТОО "Азия - Софт Сервис ЛТД"', '031040000512', '102000037, 2099-12-31 00:00:00']
['574', 'Товарищество с ограниченной ответственнос

In [228]:
pip install html5lib

     |████████████████████████████████| 112 kB 710 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
from selenium.webdriver.chrome.options import Options

def extracting_name_iin(link):
    
    soup = BeautifulSoup(urllib.request.urlopen(link).read(),'lxml')

    table = soup.find_all('table')
    df1 = pd.read_html(str(table))[2].drop([1,1]).T.drop([0,0]).reset_index()
    df2 = pd.read_html(str(table))[3]
    extracted_col = df2["Полный адрес(рус)"]

    new_dataframe = pd.concat([df1, extracted_col.rename("Address")], axis=1).drop(['index'],axis=1)
    return new_dataframe


soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup('table')[0]
final_df = pd.read_html(str(table))[0]

for row in table.find_all('tr'):
    for link in row.find_all('a', href=True):
        df = extracting_name_iin(link['href'])
        display(df)

final_df                           

,0,2,Address
0,870811351023,КОЖАНТАЕВ АЙДАР АСКАРОВИЧ,"Северо-Казахстанская область, г.Петропавловск,..."


,0,2,Address
0,760315301474,ХИЛАЖЕВ АНВАР ОЛЕГОВИЧ,"Алматинская область, Талгарский район, г.Талга..."


,0,2,Address
0,810315300428,МАКСИМОВ ЕВГЕНИЙ ИГОРЕВИЧ,"г.Астана, район Есиль, Кунаева , 12/1"
1,NaN,NaN,"г.Астана, ИМАНБАЕВОЙ, 8/1"


,0,2,Address
0,790908300861,ЖАНИБЕКОВ НУРЛАН СЕЙЛХАНОВИЧ,"г.Алматы, Медеуский район, мкр. Алатау, Ибраги..."
1,NaN,NaN,"г.Алматы, п.Алатау-Ибрагимова(Б.Ленина), 9"


,0,2,Address
0,810515350203,ЧИРЬЕВ ЕВГЕНИЙ ВАДИМОВИЧ,"г.Астана, Жилой массив КОКТАЛ , Улица БОЛАШАК, 34"
1,NaN,NaN,"г.Астана, район Сарыарка, Болашақ, 34"


...

,0,2,Address
0,910420350743,СМАГУЛОВ ЕРБОЛ ЖАНБЫРУЛЫ,"г.Астана, Проспект МАНГИЛИК ЕЛ, 8, блок 17В, о..."


,0,2,Address
0,640318400038,МАКПЫРОВА ЛАЗЗАТ МАКПЫРОВНА,"г.Алматы, Ауэзовский район, Толе би, 299, Офис..."
1,NaN,NaN,"г.Алматы, МИКРОРАЙОН АКСАЙ-3, 33"


,0,2,Address
0,800825302742,ДУРАС АЛЕКСАНДР НИКОЛАЕВИЧ,"Алматинская область, Илийский район, с.Отеген ..."
1,NaN,NaN,"Алматинская область, Илийский район, с.Отеген ..."


,0,2,Address
0,660516350117,ГУСАКОВ АЛЕКСАНДР АЛЕКСАНДРОВИЧ,"Северо-Казахстанская область, г.Петропавловск,..."
1,NaN,NaN,"Северо-Казахстанская область, г.Петропавловск,..."


ValueError: No tables found